In [27]:
import numpy as np
import pandas as pd

In [28]:
match_df = pd.read_csv("matchapps.csv")
super_df = pd.read_csv("superapps.csv")

In [29]:
DEBUG=False

# Data Cleaning

Things I've noticed:
- Robots scoring on reef while not being marked for leave
- Crazy amounts scored in some matches

In [30]:
# Robots scoring on reef while not being marked for leave
# TODO

In [31]:
# Robots scoring crazy amounts, imposing reasonable defaults
match_df[
    (match_df["autoCoral.L1"] > 5) |
    (match_df["autoCoral.L2"] > 5) |
    (match_df["autoCoral.L3"] > 5) |
    (match_df["autoCoral.L4"] > 5) |
    (match_df["teleCoral.L1"] > 10) |
    (match_df["teleCoral.L2"] > 10) |
    (match_df["teleCoral.L3"] > 10) |
    (match_df["teleCoral.L4"] > 10)
][
    ["metadata.robotTeam", "metadata.matchNumber"] +
    ["autoCoral.L{}".format(x) for x in range(1, 5)] +
    ["teleCoral.L{}".format(x) for x in range(1, 5)]
]


,metadata.robotTeam,metadata.matchNumber,autoCoral.L1,autoCoral.L2,autoCoral.L3,autoCoral.L4,teleCoral.L1,teleCoral.L2,teleCoral.L3,teleCoral.L4
335,3863.0,43,0,0,0,0,3,0,22,0


In [32]:
# Index 335 - update values to:
# 4 L1
# 8 L3

match_df.loc[335, "teleCoral.L1"] = 4
match_df.loc[335, "teleCoral.L3"] = 8

match_df[match_df.index == 335][
    ["metadata.robotTeam", "metadata.matchNumber"] +
    ["autoCoral.L{}".format(x) for x in range(1, 5)] +
    ["teleCoral.L{}".format(x) for x in range(1, 5)]
]

,metadata.robotTeam,metadata.matchNumber,autoCoral.L1,autoCoral.L2,autoCoral.L3,autoCoral.L4,teleCoral.L1,teleCoral.L2,teleCoral.L3,teleCoral.L4
335,3863.0,43,0,0,0,0,4,0,8,0


# Adding New Fields

In [33]:
# Grab a random sample match
DEBUG and match_df[(match_df["metadata.robotTeam"] == 4201) & (match_df["metadata.matchNumber"] == 8)]

False

In [34]:
# Add columns for auto, tele, and endgame score
match_df["autoScore"] = (
    match_df["leftStartingZone"] * 3 +
    match_df["autoCoral.L1"] * 3 +
    match_df["autoCoral.L2"] * 4 +
    match_df["autoCoral.L3"] * 6 +
    match_df["autoCoral.L4"] * 7 +
    match_df["autoAlgae.processor"] * 6 +
    match_df["autoAlgae.netRobot"] * 4
)

match_df["teleScore"] = (
    match_df["teleCoral.L1"] * 2 +
    match_df["teleCoral.L2"] * 3 +
    match_df["teleCoral.L3"] * 4 +
    match_df["teleCoral.L4"] * 5 +
    match_df["teleAlgae.processor"] * 6 +
    match_df["teleAlgae.netRobot"] * 4
)

match_df["climb.park"] = match_df["climb"] == "park"
match_df["climb.shallow"] = match_df["climb"] == "shallow"
match_df["climb.deep"] = match_df["climb"] == "deep"

match_df["endgameScore"] = (
    match_df["climb.park"] * 2 +
    match_df["climb.shallow"] * 6 +
    match_df["climb.deep"] * 12
)

match_df["totalScore"] = (
    match_df["autoScore"] +
    match_df["teleScore"] +
    match_df["endgameScore"]
)


In [35]:
# Grab a random sample match to verify
DEBUG and match_df[match_df["metadata.robotTeam"] == 3255][["autoCoral.L1", "autoCoral.L2", "autoCoral.L3", "autoCoral.L4", "autoAlgae.processor", "autoAlgae.netRobot", "autoScore"]]

False

In [36]:
# Grab a random sample match to verify
DEBUG and match_df[match_df["metadata.robotTeam"] == 3255][["leftStartingZone", "teleCoral.L1", "teleCoral.L2", "teleCoral.L3", "teleCoral.L4", "teleAlgae.processor", "teleAlgae.netRobot", "teleScore"]]

False

In [37]:
# Grab a random sample match to verify
DEBUG and match_df[match_df["metadata.robotTeam"] == 3255][["climb.park", "climb.shallow", "climb.deep", "endgameScore"]]

False

In [38]:
# Grab a random sample match to verify
DEBUG and match_df[match_df["metadata.robotTeam"] == 3255][["autoScore", "teleScore", "endgameScore", "totalScore"]]

False

In [39]:
# Add columns for whether teams have scored on the side of the reef in auto
match_df["metadata.robotColorIsBlue"] = (
    (match_df["metadata.robotPosition"] == "blue_1") |
    (match_df["metadata.robotPosition"] == "blue_2") |
    (match_df["metadata.robotPosition"] == "blue_3")
)

# Calculate if a team scored on the right side of the reef, from the perspective of the barge
match_df["autoScoreRightReef"] = (
    match_df["metadata.robotColorIsBlue"] & (match_df["placement.deposit1"] | match_df["placement.deposit6"]) |
    ~match_df["metadata.robotColorIsBlue"] & (match_df["placement.deposit3"] | match_df["placement.deposit4"])
)

# Calculate if a team scored on the left side of the reef, from the perspective of the barge
match_df["autoScoreLeftReef"] = (
    match_df["metadata.robotColorIsBlue"] & (match_df["placement.deposit3"] | match_df["placement.deposit4"]) |
    ~match_df["metadata.robotColorIsBlue"] & (match_df["placement.deposit1"] | match_df["placement.deposit6"])
)

In [40]:
# Grab a random sample match to verify
DEBUG and match_df[match_df["autoScoreRightReef"] | match_df["autoScoreLeftReef"]][["metadata.matchNumber", "metadata.robotTeam", "metadata.robotPosition", "placement.deposit1", "placement.deposit2", "placement.deposit3", "placement.deposit4", "placement.deposit5", "placement.deposit6", "autoScoreRightReef", "autoScoreLeftReef"]]

False

In [41]:
team_df = pd.DataFrame()

In [42]:
# For each of the columns below, find only the average
AVG_COLUMNS = (
    ["startingZone.start{}".format(x) for x in range(1, 4)] +
    ["pickupLocation.source{}".format(x) for x in range(1, 3)] +
    ["pickupLocation.ground{}".format(x) for x in range(1, 4)] +
    ["placement.deposit{}".format(x) for x in range(1, 7)] +
    ["climb.park", "climb.shallow", "climb.deep", "autoScoreRightReef", "autoScoreLeftReef"]
)

for c in AVG_COLUMNS:
    team_df[f"{c}.avg"] = match_df.groupby("metadata.robotTeam")[c].mean()

In [43]:
# For each of the columns below, find statistical metrics for each (min, max, avg, std)
STAT_COLUMNS = (
    ["autoCoral.L{}".format(x) for x in range(1, 5)] +
    ["teleCoral.L{}".format(x) for x in range(1, 5)] +
    ["autoAlgae.netRobot", "autoAlgae.processor", "autoAlgae.remove"] +
    ["teleAlgae.netRobot", "teleAlgae.processor", "teleAlgae.remove"] +
    ["autoScore", "teleScore", "endgameScore", "totalScore"]
)

for c in STAT_COLUMNS:
    team_df[f"{c}.min"] = match_df.groupby("metadata.robotTeam")[c].min()
    team_df[f"{c}.max"] = match_df.groupby("metadata.robotTeam")[c].max()
    team_df[f"{c}.avg"] = match_df.groupby("metadata.robotTeam")[c].mean()
    team_df[f"{c}.std"] = match_df.groupby("metadata.robotTeam")[c].std()

    # We also want to calculate these metrics for different variations of matches being dropped.
    for n in range(1, 3):
        # Start by removing the top and bottom `n` matches for a team by finding
        # the indexes of the matches we want to keep
        indexes_to_keep = match_df.groupby("metadata.robotTeam").apply(lambda x: x.sort_values(c).iloc[n:-n]).index.get_level_values(1)
        filtered_df = match_df[match_df.index.isin(indexes_to_keep)]

        # With our new filtered dataframe, compute statistic metrics
        team_df[f"{c}.min.drop{n}"] = filtered_df.groupby("metadata.robotTeam")[c].min()
        team_df[f"{c}.max.drop{n}"] = filtered_df.groupby("metadata.robotTeam")[c].max()
        team_df[f"{c}.avg.drop{n}"] = filtered_df.groupby("metadata.robotTeam")[c].mean()
        team_df[f"{c}.std.drop{n}"] = filtered_df.groupby("metadata.robotTeam")[c].std()

/tmp/ipykernel_192619/2394994711.py:20: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  indexes_to_keep = match_df.groupby("metadata.robotTeam").apply(lambda x: x.sort_values(c).iloc[n:-n]).index.get_level_values(1)
/tmp/ipykernel_192619/2394994711.py:20: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  indexes_to_keep = match_df.groupby("metadata.robotTeam").apply(lambda x: x.sort_values(c).iloc[n:-n]).index.ge

/tmp/ipykernel_192619/2394994711.py:20: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  indexes_to_keep = match_df.groupby("metadata.robotTeam").apply(lambda x: x.sort_values(c).iloc[n:-n]).index.get_level_values(1)
/tmp/ipykernel_192619/2394994711.py:20: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  indexes_to_keep = match_df.groupby("metadata.robotTeam").apply(lambda x: x.sort_values(c).iloc[n:-n]).index.ge

In [44]:
DEBUG and team_df[team_df.index == 4201][["autoCoral.L4.avg", "autoCoral.L4.avg.drop1", "autoCoral.L4.avg.drop2"]]

False

In [45]:
DEBUG and match_df[match_df["metadata.robotTeam"] == 4201][["metadata.robotTeam", "autoCoral.L4"]]

False

In [46]:
team_df["fouls.avg"] = super_df[[
    "fouls.insideRobot",
    "fouls.protectedZone",
    "fouls.pinning",
    "fouls.multiplePieces",
    "fouls.cageFoul",
    "fouls.other"
]].sum(axis=1).groupby(super_df["metadata.robotTeam"]).mean()

/tmp/ipykernel_192619/1034481741.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  team_df["fouls.avg"] = super_df[[


In [47]:
DEBUG and team_df["fouls.avg"]

False

In [48]:
team_df["break.avg"] = super_df[[
    "break.mechanismDmg",
    "break.batteryFall",
    "break.commsFail",
    "break.bumperFall"
]].sum(axis=1).groupby(super_df["metadata.robotTeam"]).mean()

/tmp/ipykernel_192619/634828927.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  team_df["break.avg"] = super_df[[


In [49]:
DEBUG and team_df["break.avg"]

False

In [50]:
# All copied from last year, didn't actually check if this works
def safeDivide(a, b):
    return 0 if b == 0 else a / b

team_list = match_df["metadata.robotTeam"].unique().tolist()
team_list = [team for team in team_list if not np.isnan(team)]

team_df["Percent of Matches with No Defense"] = None
team_df["Percent of Matches with Some Defense"] = None
team_df["Percent of Matches with Full Defense"] = None
team_df["Main Defense Type"] = None

for team in team_list:
    
    defense_type_list = super_df.loc[super_df["metadata.robotTeam"] == team, "defense"].tolist()
    
    no_defense_count = 0
    some_defense_count = 0
    full_defense_count = 0
    
    for defense_type in defense_type_list:
        if defense_type == "noDef":
            no_defense_count += 1
        elif defense_type == "someDef":
            some_defense_count += 1
        elif defense_type == "fullDef":
            full_defense_count += 1

    defense_total_count = sum([no_defense_count, some_defense_count, full_defense_count])
        
    team_df.at[team, "Percent of Matches with No Defense"] = safeDivide(no_defense_count, defense_total_count)
    team_df.at[team, "Percent of Matches with Some Defense"] = safeDivide(some_defense_count, defense_total_count)
    team_df.at[team, "Percent of Matches with Full Defense"] = safeDivide(full_defense_count, defense_total_count)
    
    biggest_defense_count = max([no_defense_count, some_defense_count, full_defense_count])
    main_defense_type = ""

    if biggest_defense_count == no_defense_count:
        main_defense_type += "No Defense "
    if biggest_defense_count == some_defense_count:
        main_defense_type += "Some Defense "
    if biggest_defense_count == full_defense_count:
        main_defense_type += "Full Defense"
    
    team_df.at[team, "Main Defense Type"] = main_defense_type

/tmp/ipykernel_192619/1343473457.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  team_df["Percent of Matches with No Defense"] = None
/tmp/ipykernel_192619/1343473457.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  team_df["Percent of Matches with Some Defense"] = None
/tmp/ipykernel_192619/1343473457.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.

In [51]:
DEBUG and team_df

False

In [52]:
team_df.to_csv("caph_2025_sat.csv")